In [1]:
# External libraries
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
from sklearn import linear_model, datasets
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

style.use('seaborn-paper')
%matplotlib tk

global km2px, deg2km, px2km, deg2px

def find_pos(idx, TOL1, TOL2, TOL3, TOL4):
    # Loading All Images:
    dict = load_all_images(dt=10)
    # Img:
    filename = dict[str(idx+1)]
    img=cv2.imread(filename)
    # Detection:
    t1 = time.time()
    craters_det = detect(img)
    # Removing minor craters:
    craters_det = craters_det[craters_det[:,2] > 15]
    t2 = time.time()
    print(f'Detection Time:{t2-t1:.2f}\n')
    # Pandas DataFrame:
    df_craters_det = sort_mat(craters_det)
    # Find all triplets:
    t1 = time.time()
    triplets = find_all_triplets(craters_det)
    triplets_det= pd.DataFrame(triplets, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3',   'y3','r3'])
    triplets_det.shape
    t2 = time.time()
    print('\n')
    print(f'Total craters founded:{craters_det.shape[0]}')
    print(f'Number of total combinations:{triplets_det.shape[0]}\nComputational time: {t2-t1:.2f} s')

###############################################################################################################
    # Opening Database:
    # DB = pd.read_csv('DATA/H_L_combined.csv')
    DB = pd.read_csv('DATA/lunar_crater_database_robbins_2018.csv')

    # Filtering DATABASE:
    MULTIPLAYER = 1.
    span = 3.29/2   *MULTIPLAYER # TODO: Aggiustare l'immagine di catalogo a grandezza effettiva!
    lat_bounds=[-span, span]
    get_lon = float(filename.split('_')[-1].split('jpg')[0][:-2])
    lon_bounds=[get_lon-span,get_lon+span]
    # craters_cat = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME='COMBINED')
    craters_cat = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME='ROBBINS')

    # FIND TRIPLETS:
    if craters_cat is not None:
        km2deg = 1/deg2km
        craters_cat = craters_cat[(craters_cat.Diam < 35)&(craters_cat.Diam > 4)]
        craters_cat['Diam']*=0.5*km2deg # km --- > deg

        craters_cat_m = np.array(craters_cat)
        t1 = time.time()
        triplets_cat_m = find_all_triplets(craters_cat_m)
        triplets_cat = pd.DataFrame(triplets_cat_m, columns=['Angle1','Angle2','Angle3','des1','des2','des3','lon1','lat1', 'r1','lon2',    'lat2','r2','lon3','lat3','r3'])
        triplets_cat['r1'] *= deg2km
        triplets_cat['r2'] *= deg2km
        triplets_cat['r3'] *= deg2km
        t2 = time.time()
        print(f'Total craters catalogued:{craters_cat.shape[0]+1}')
        print(f'Number of total combinations:{triplets_cat.shape[0]}\nComputational time: {t2-t1:.2f} s')
    else:
        print('No craters in cat!')
###############################################################################################################
    tol1 = TOL1

    t1 = time.time()
    QUERY1 = triplets_cat
    QUERY2 = triplets_det
    QUERY1 = dropduplicates(QUERY1)
    QUERY2 = dropduplicates(QUERY2) 

    if QUERY1.shape[0]<QUERY2.shape[0]:
        mode = 'natural'
        joins, items = inner_join(QUERY1, QUERY2, tol1)
    else:
        mode = 'inverse'
        joins, items = inner_join(QUERY2, QUERY1, tol1)
    print(f'Mode:{ mode}')
    t2 = time.time()
    print(f'Computational time: {t2-t1:.2f} s\nPossible list Combinations: {len(items)}')
###############################################################################################################
    t1 = time.time()
    tol2 = TOL2
    S, iss = [], []
    for i in range(len(joins)):
        join = joins[i]
        des1, des2, des3 = items[i].des1, items[i].des2, items[i].des3
        s=join[ (abs(join.des1 - des1) < tol2) & (abs(join.des2 - des2) < tol2) & (abs(join.des3 - des3) < tol2)\
              | (abs(join.des1 - des2) < tol2) & (abs(join.des2 - des3) < tol2) & (abs(join.des3 - des1) < tol2)\
              | (abs(join.des1 - des3) < tol2) & (abs(join.des2 - des1) < tol2) & (abs(join.des3 - des2) < tol2)]

        if s.shape[0] > 0:
            S.append(s)
            iss.append(items[i])
    t2 = time.time()
    print(f'Computational time: {t2-t1:.2f} s\nPossible list Combinations: {len(S)}')
###############################################################################################################
    # FIND TRUE COMBINATIONS:
    CAMx, CAMy = ( (lon_bounds[0] + lon_bounds[1]) / 2, (lat_bounds[0] + lat_bounds[1]) / 2) # Location Absolute

    Is, Js = [], []
    for I in range(len(iss)):
        row1 = iss[I]
        for J in range(S[I].shape[0]):
            if check_sol(I,J, TOL3, mode, S, iss) & check_sol2(I,J, TOL4, mode, S, iss, CAMx, CAMy): # TODO: Aggiungere checksol3:   confronto crts
                    Is.append(I)
                    Js.append(J)
                    print(I,J)
    Is = np.array(Is)
    Js = np.array(Js)
###############################################################################################################
    if len(Js)!=0:    

        H, px2deg = H_estimation(Is, Js, mode, S, iss, CAMx, CAMy)
        print(f'Height: {H:.2f}          px2deg: {px2deg:.5f}')

        LON, LAT = [], []
        for s in range(len(Is)):
            i, j = Is[s], Js[s]
            A,B,B_a = find_ABBa(i,j,mode, S, iss, CAMx, CAMy)

            Lon, Lat = LL_estimation(A, B, B_a, px2deg)
            LON.append(Lon)
            LAT.append(Lat)
        if len(LON)>3:
            LON_m = np.mean(filter_quartile(LON))
            LAT_m = np.mean(filter_quartile(LAT))
        else:
            LON_m = np.mean(LON)
            LAT_m = np.mean(LAT)

        print(f'LON: {LON_m:.3f}       LAT:{LAT_m:.3f}')
    else: print('Association was not possible...')
###############################################################################################################
#                                                    READ REAL DATA:
###############################################################################################################
    dt = 10
    df = pd.read_csv(f'DATA/ephemeris sat/inclination zero/{dt} step size.csv', header=3, sep=';') 
    real_Latitudes, real_Longitudes, real_Altitudes = df['Lat (deg)'], df['Lon (deg)'], df['Alt (km)']
    real_Vxs,real_Vys,real_Vzs = df['x (km/sec)'], df['y (km/sec)'],df['z (km/sec)']

    real_X, real_Y, real_Z = [], [], []
    for i in range(len(df)):
        altitude = real_Altitudes[i]
        latitude = real_Latitudes[i]
        longitude = real_Longitudes[i]
        x, y, z = spherical2cartesian(altitude, latitude, longitude)
        real_X.append(x)
        real_Y.append(y)
        real_Z.append(z)
    real_X, real_Y, real_Z = np.array(real_X),np.array(real_Y),np.array(real_Z)
#############################################################################################################################

    Error = []
    if len(Js)!=0: 
        TRUE_POS = np.array([real_Longitudes[idx], real_Latitudes[idx]])
        # print(TRUE_POS)
        D = TRUE_POS - np.array([LON_m,LAT_m])
        ERROR_LL = D*deg2km
        Error.append(H-50)
        Error.append(ERROR_LL[0])
        Error.append(ERROR_LL[1])
        print(f'Error (km)| H: {Error[0]:.2f}Km    Lon:{Error[1]:.2f}Km    Lat:{Error[2]:.2f}Km')

        x,y,z = spherical2cartesian(H, LAT_m, LON_m)
        x,y,z = x*1000, y*1000, z*1000 # Km ---> m

        xEast, yNorth, zUp = ecef_to_enu(x, y, z, real_Latitudes[idx], real_Longitudes[idx], real_Altitudes[idx]*1000)
        print(f'ENU frame| xEast: {xEast:.2f}m    yNorth:{yNorth:.2f}m    zUp:{zUp:.2f}m')

    else: print('Association was not possible...')

    if len(Js)!=0: 
        Position_est = np.array([H, LAT_m, LON_m])
        Error_est = np.array([abs(Error[0]),abs(Error[1]),abs(Error[2])])
        ENU = np.array([abs(xEast),abs(yNorth),abs(zUp)])
        return Position_est, Error_est, ENU
    else:
        return None 


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
FILE = pd.DataFrame(columns=['ID','H','LAT','LON','Error_H(km)','Error_LAT(km)','Error_LON(km)','xEast','yNorth','zUp'])

# a = np.linspace(0,615,616).astype('int')
# b = np.linspace(624,678,678-624+1).astype('int')
# indexes = np.hstack([a,b])

T1, T2, T3, T4 = 5, 0.9, 0.1, 0.1


for i in range(678):
    try:
        pos , err, ENU = find_pos(i, T1, T2, T3, T4)
        new_row = {'ID':i, 'H':pos[0], 'LAT':pos[1],'LON':pos[2],'Error_H(km)':err[0],'Error_LAT(km)':err[1],\
                    'Error_LON(km)':err[2], 'xEast':ENU[0],'yNorth':ENU[1],'zUp':ENU[2]}
        FILE = FILE.append(new_row, ignore_index=True)
        FILE.to_csv(f'Pos_Err_d100_quartile_40-60_tol1={T1}_tol2={T2}_t3={T3}_t4={T4}.csv')

    except TypeError: 
        print('Sorry, cannot find matches...')
        new_row = {'ID':i, 'H':np.NaN, 'LAT':np.NaN,'LON':np.NaN,'Error_H(km)':np.NaN,'Error_LAT(km)':np.NaN,\
                    'Error_LON(km)':np.NaN, 'xEast':np.NaN,'yNorth':np.NaN,'zUp':np.NaN}
        FILE = FILE.append(new_row, ignore_index=True)
        FILE.to_csv(f'Pos_Err_d100_quartile_40-60_tol1={T1}_tol2={T2}_t3={T3}_t4={T4}.csv')

    except UnboundLocalError: 
        print('Sorry, no crater in catalog..')
        new_row = {'ID':i, 'H':np.NaN, 'LAT':np.NaN,'LON':np.NaN,'Error_H(km)':np.NaN,'Error_LAT(km)':np.NaN,\
                    'Error_LON(km)':np.NaN, 'xEast':np.NaN,'yNorth':np.NaN,'zUp':np.NaN}
        FILE = FILE.append(new_row, ignore_index=True)
        FILE.to_csv(f'Pos_Err_d100_quartile_40-60_tol1={T1}_tol2={T2}_t3={T3}_t4={T4}.csv')

    except NameError: 
        print('Sorry, skipping this image..')
        new_row = {'ID':i, 'H':np.NaN, 'LAT':np.NaN,'LON':np.NaN,'Error_H(km)':np.NaN,'Error_LAT(km)':np.NaN,\
                    'Error_LON(km)':np.NaN, 'xEast':np.NaN,'yNorth':np.NaN,'zUp':np.NaN}
        FILE = FILE.append(new_row, ignore_index=True)
        FILE.to_csv(f'Pos_Err_d100_quartile_40-60_tol1={T1}_tol2={T2}_t3={T3}_t4={T4}.csv')
    
    except KeyError:
        print('This image is missing, adding null row..')
        new_row = {'ID':i, 'H':np.NaN, 'LAT':np.NaN,'LON':np.NaN,'Error_H(km)':np.NaN,'Error_LAT(km)':np.NaN,\
                    'Error_LON(km)':np.NaN, 'xEast':np.NaN,'yNorth':np.NaN,'zUp':np.NaN}
        FILE = FILE.append(new_row, ignore_index=True)
        FILE.to_csv(f'Pos_Err_d100_quartile_40-60_tol1={T1}_tol2={T2}_t3={T3}_t4={T4}.csv')

print('Done!')

9.253       LAT:-0.000
Error (km)| H: 0.27Km    Lon:0.13Km    Lat:-0.01Km
ENU frame| xEast: -132.02m    yNorth:12.69m    zUp:273.96m
Detection Time:0.50

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


Total craters founded:23
Number of total combinations:10626
Computational time: 0.16 s
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Total craters catalogued:52
Number of total combinations:124950
Computational time: 1.81 s
Mode:inverse
Computational time: 16.06 s
Possible list Combinations: 1771
Computational time: 6.28 s
Possible list Combinations: 1088
71 2
92 0
178 2
179 2
181 1
202 1
203 0
204 0
211 0
214 1
217 0
219 2
223 0
226 1
262 3
267 0
275 1
280 2
283 1
287 0
294 0
297 0
298 3
303 0
311 3
336 0
348 0
349 1
361 1
385 0
392 0
502 0
520 0
527 0
549 0
580 0
601 0
603 0
615 1
627 0
656 4
659 1
661 1
664 0
682 3
688 1
693 3
729 1
735 0
758 2
759 2
768 

In [3]:
FILE = pd.read_csv(f'Pos_Err_d100_quartile_40-60_tol1={T1}_tol2={T2}_t3={T3}_t4={T4}.csv')

In [4]:
H, Lon, Lat, IDXs = FILE['Error_H(km)'], FILE['Error_LON(km)'], FILE['Error_LAT(km)'], FILE['ID']

# Altitude

In [5]:
plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, H, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('Km')
plt.title('Altitude Error')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, H, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('Km')
plt.ylim([0,2])
plt.title('Altitude Error')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335,:Img.shape[1]//2])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.savefig('AltitudeError_d=100_quartile_40-60A.jpg')
plt.show()


plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, H, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('Km')
plt.title('Altitude Error')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, H, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('Km')
plt.ylim([0,2])
plt.title('Altitude Error')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2: ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])
plt.savefig('AltitudeError_d=100_quartile_40-60B.jpg')
plt.show()

# Longitude

In [6]:
plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, Lon, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('Km')
plt.title('Longitude Error')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, Lon, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('Km')
plt.ylim([0,1])
plt.title('Longitude Error')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335,:Img.shape[1]//2])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.savefig('LongError_d=100_quartile_40-60A.jpg')
plt.show()


plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, Lon, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('Km')
plt.title('Longitude Error')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, Lon, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('Km')
plt.ylim([0,1])
plt.title('Longitude Error')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2: ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])
plt.savefig('LongError_d=100_quartile_40-60B.jpg')
plt.show()

# Latitude

In [7]:
plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, Lat, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('Km')
plt.title('Latitude Error')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, Lat, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('Km')
plt.ylim([0,1])
plt.title('Latitude Error')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335,:Img.shape[1]//2])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.savefig('LatError_d=100_quartile_40-60A.jpg')
plt.show()


plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, Lat, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('Km')
plt.title('Latitude Error')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, Lat, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('Km')
plt.ylim([0,1])
plt.title('Latitude Error')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2: ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])
plt.savefig('LatError_d=100_quartile_40-60B.jpg')
plt.show()

In [8]:
xEast, yNorth, zUp = FILE['xEast'], FILE['yNorth'], FILE['zUp']

In [9]:
def filter_quartile(Xs):
    X = pd.DataFrame(Xs)
    Q1 = X.quantile(0.2)
    Q3 = X.quantile(0.8)
    IQR = Q3 - Q1
    X = X[np.logical_not((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR)))]
    X = X.dropna()
    return np.array(X)


xEast_filtered = filter_quartile(xEast)
yNorth_filtered = filter_quartile(yNorth)
zUp_filtered = filter_quartile(zUp)
print(np.mean(xEast_filtered))
print(np.mean(yNorth_filtered))
print(np.mean(zUp_filtered))

from scipy.interpolate import make_interp_spline, BSpline

#create data
x = range(len(xEast_filtered))
y = xEast_filtered

#define x as 200 equally spaced values between the min and max of original x 
xnew = np.linspace(x[0], x[-1], 50)

#define spline
spl = make_interp_spline(x, y, k=3)
y_smooth = spl(xnew)

#create smooth line chart 
plt.plot(xnew, y_smooth)
plt.show()


194.389371288703
171.15543155267108
289.09372584450443


# East

In [10]:
plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, xEast, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('m')
plt.title('East')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, xEast, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('m')
plt.ylim([0,1000])
plt.title('East')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335,:Img.shape[1]//2])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.savefig('EastError_d=100_quartile_40-60A.jpg')
plt.show()


plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, xEast, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('m')
plt.title('East')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, xEast, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('m')
plt.ylim([0,1000])
plt.title('East')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2: ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])
plt.savefig('EastError_d=100_quartile_40-60B.jpg')
plt.show()

# North

In [11]:
plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, yNorth, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('m')
plt.title('North')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, yNorth, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('m')
plt.ylim([0,1000])
plt.title('North')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335,:Img.shape[1]//2])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.savefig('NorthError_d=100_quartile_40-60A.jpg')
plt.show()


plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, yNorth, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('m')
plt.title('North')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, yNorth, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('m')
plt.ylim([0,1000])
plt.title('North')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2: ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])
plt.savefig('NorthError_d=100_quartile_40-60B.jpg')
plt.show()

# Up

In [12]:
plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, zUp, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('m')
plt.title('Up')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, yNorth, s =2)
plt.xlim([0,end//2])
plt.xlabel('idx')
plt.ylabel('m')
plt.ylim([0,1500])
plt.title('Up')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335,:Img.shape[1]//2])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.savefig('UpError_d=100_quartile_40-60A.jpg')
plt.show()


plt.figure(dpi=150, figsize=(10,6), tight_layout=True)
end = len(IDXs)
plt.subplot(311)
plt.scatter(IDXs, zUp, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('m')
plt.title('Up')
plt.show()

plt.subplot(312)
plt.scatter(IDXs, yNorth, s =2)
plt.xlim([end//2, end])
plt.xlabel('idx')
plt.ylabel('m')
plt.ylim([0,1500])
plt.title('Up')

plt.subplot(313)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2: ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])
plt.savefig('UpError_d=100_quartile_40-60B.jpg')
plt.show()